Problema 4.

En primer lugar, github no permite subir archivos de mas de 25 mb, por lo tanto no se pueden subir los batches.
Asi que le pido por favor que descarge el notebook y ponga los batches (supongo que los debe tener) en una carpeta llamada
data y que este al lado del notebook, gracias.

Para el desarrollo del modelo, que permita clasificar las imágenes de CIFAR10, se tomó en consideración
cada uno de los parámetros estudiados en el problema 3. Antes de realizar a implementación, se estudiaron 
diversos modelos que permiten una buena clasificación de imagenes, pero la mayoría, por no decir todos, realizaban
sus redes en base a redes convolusionales, lo cual parece ser o mejor en este caso.
Como es necesario, por ahora, realizar redes feed forward, se utilizaron los parámetros, que mejor resultado 
mostraron en el problema anterior, es decir, cada parámetro junto al valor, que según lo observado, influía en una
mejor y más rápida convergencia.
Es por eso que, además de realizar un escalamiento de los datos, se utilizaron factores de Weight decay, un learning rate bajo, un factor
de dropout en cada una de las capas ocultas donde cada una tenía una función de activación RELU, función que mostró
mejores resultados en comparación a SIGMOID. 
Se utilizó además, un optimizador SGD, que nuevamente, en comparación a los otros, 
dió resultados de convergencia más rápido. (lr=0.001, decay=1e-6, momentum = 0.8) Este optimizador, utilizó un learning rate de 0.001,
uno no tan chico, ni tan grande, un learning decay de 1e-6 y un momentum de 0,8. No se utilizo el momento clásico, puesto que no se
demostró una gran mejoría.
Se realizaron diversas pruebas utilizando para ello, un conjunto de validación de 10000 para training, y 3000 de testing. Después 
de algunas pruebas, se llega a la conclusión que, es necesario darle un gran número de capas ocultas, para que así se logre un mayor
nivel de abstracción con las imágenes. Esto puede ser ya que de por si, las imagenes, al trabajar de pixel a pixel, tienen un nivel
de no linealidad muy alta. Por lo que se le aumentó significativamente el número de estas, utilizando finalmente, 10 capas ocultas.
Para la cantidad de epochs, se pensó que quizás un número alto, permetiría obtener un mejor resultado, lamentablemente no fue así. Pero 
finalmente, se le asignó un total de 700 epochs.

Para el proceso de entrenamiento final, el modelo se implementó en un servidor, el cual gracias a la empresa donde uno de 
nosotros trabaja. Hacia este servidor, se puede acceder vía ssh, por lo que se configuró el ambiente, para que se puedan entrenar redes
utilizando python y keras. Una vez listo, se realizó un pequeño script, para que la red se mantenga entrenando, aún después de 
que el usuario cerrara la sesión. 
El entrenamiento total, tomó alrededor de 4 días. Cerca del primer día, la red comenzó a tener overfitting, es decir, el valor de error
en el entrenamiento comenzó a ser muy bajo, y el de accuracy en entrenamiento, casi cercano a 1 (100%), pero no así en el caso de los 
valores de testeo, estos comenzaron a estancarse en un 60% de exactitud.





In [ ]:
from __future__ import print_function
import cPickle as pickle
import numpy as np
import os
from scipy.misc import imread
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv1D,Conv2D, MaxPooling2D
from keras.layers.core import Dense, Activation
from keras.optimizers import SGD
import matplotlib
# Force matplotlib to not use any Xwindows backend.
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import pandas as pd

label_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog',\
'frog', 'horse', 'ship', 'truck']


def load_CIFAR_one(filename):
	with open(filename, 'rb') as f:
		datadict = pickle.load(f)
		X = datadict['data']
		Y = datadict['labels']
		Y = np.array(Y)
		return X, Y

def load_CIFAR10(PATH):
	xs = []
	ys = []
	for b in range(1,6):
		f = os.path.join(PATH, 'data_batch_%d' % (b, ))
		X, Y = load_CIFAR_one(f)
		xs.append(X)
		ys.append(Y)
	Xtr = np.concatenate(xs)
	Ytr = np.concatenate(ys)
	del X, Y
	Xte, Yte = load_CIFAR_one(os.path.join(PATH, 'test_batch'))
	return Xtr, Ytr, Xte, Yte

Xtr,Ytr,Xte,Yte = load_CIFAR10('data/')

Xva = Xtr[0:10000].copy()
Yva = Ytr[0:10000].copy()



batch_size = 32
num_classes = 10
epochs = 700


# Convert class vectors to binary class matrices.
#Ytr = keras.utils.to_categorical(Ytr, num_classes)
#Yte = keras.utils.to_categorical(Yte, num_classes)

Xtr = Xtr.astype('float32')
Xte= Xte.astype('float32')


from sklearn.preprocessing import StandardScaler
from keras.regularizers import l1
from keras.layers import Dropout

scaler = StandardScaler().fit(Xtr)
Xtrs = pd.DataFrame(scaler.transform(Xtr))
#Xtes = pd.DataFrame(scaler.transform(Xte))

scalerVa = StandardScaler().fit(Xva)
Xvas = pd.DataFrame(scalerVa.transform(Xva))
scalerVa = StandardScaler().fit(Xte)
Xtes = pd.DataFrame(scaler.transform(Xte))
 
# Convert class vectors to binary class matrices.
Ytr = keras.utils.to_categorical(Ytr, num_classes)
Yte = keras.utils.to_categorical(Yte, num_classes)
Yva = keras.utils.to_categorical(Yva, num_classes)

regu = 0.00001

model = Sequential()

model.add(Dense(2000, activation="relu", kernel_initializer="uniform", input_dim=Xtr.shape[1],W_regularizer=l1(regu)))
model.add(Dropout(0.4))
model.add(Dense(1200, activation="relu", kernel_initializer="uniform",W_regularizer=l1(regu)))
model.add(Dropout(0.25))
model.add(Dense(1200, activation="relu", kernel_initializer="uniform",W_regularizer=l1(regu)))
model.add(Dropout(0.25))
model.add(Dense(1200, activation="relu", kernel_initializer="uniform",W_regularizer=l1(regu)))
model.add(Dropout(0.25))
model.add(Dense(1200, activation="relu", kernel_initializer="uniform",W_regularizer=l1(regu)))
model.add(Dropout(0.25))
model.add(Dense(1200, activation="relu", kernel_initializer="uniform",W_regularizer=l1(regu)))
model.add(Dropout(0.25))
model.add(Dense(1200, activation="relu", kernel_initializer="uniform",W_regularizer=l1(regu)))
model.add(Dropout(0.25))
model.add(Dense(1200, activation="relu", kernel_initializer="uniform",W_regularizer=l1(regu)))
model.add(Dropout(0.25))
model.add(Dense(1200, activation="relu", kernel_initializer="uniform",W_regularizer=l1(regu)))
model.add(Dropout(0.25))
model.add(Dense(512, activation="relu", kernel_initializer="uniform",W_regularizer=l1(regu)))
model.add(Dropout(0.5))
model.add(Dense(units=10, activation="softmax"))


sgd = SGD(lr=0.001, decay=1e-6, momentum = 0.8)
model.compile(optimizer=sgd ,loss='categorical_crossentropy',metrics=['accuracy'])

hist = model.fit(Xtrs.as_matrix(), Ytr, epochs=epochs,batch_size=batch_size, validation_data=(Xtes.as_matrix(), Yte))

print(hist.history.keys())

# summarize history for loss
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['loss train', 'loss test'], loc='upper left')
plt.savefig('lossProblema4.jpg')

plt.clf()
plt.plot(hist.history['acc'])
plt.plot(hist.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('acc')
plt.xlabel('epoch')
plt.legend(['acc train','acc test'], loc='upper left')
plt.savefig('accProblema4.jpg')

Lamentablemente no se logró entrenar la red, con la mejora de atributos expuestas en el punto d), por lo que no sé logró realizar
una comparación entre ambos resultados. Aún así, se sub entiende, que al mejorar los parámetros que utiliza el modelo, se deberían 
lograr mejores resultados con la red.
